In [1]:
!gdown 1sCTQXp9kTcm8_jckqC-v_gWxIsFCcUbS
!gdown 1vK24A09o5Nev5qj1qNhndFe6beTWSDRU
!gdown 1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2

Downloading...
From: https://drive.google.com/uc?id=1sCTQXp9kTcm8_jckqC-v_gWxIsFCcUbS
To: /content/X_train
100% 7.14M/7.14M [00:00<00:00, 41.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vK24A09o5Nev5qj1qNhndFe6beTWSDRU
To: /content/Y_train
100% 65.1k/65.1k [00:00<00:00, 42.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2
To: /content/X_test
100% 3.57M/3.57M [00:00<00:00, 78.2MB/s]


In [2]:
import numpy as np
import pandas as pd
import csv
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.decomposition import PCA

In [3]:
def normalize(X, mu_x=None, std_x=None):

  #ToDo Begin

  if mu_x is None and std_x is None:
    mu_x = np.mean(X, axis=0)
    std_x = np.std(X, axis=0)

  #ToDo End
  X = (X - mu_x) / std_x
  return X, mu_x, std_x

def load_train():
  X = pd.read_csv("X_train")
  X_c = X[['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']].values
  X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss'], 1).iloc[:, :-1].values
  Y = pd.read_csv("Y_train", header = None).values.reshape(-1)
  X_c, mu, std = normalize(X_c)
  X = np.concatenate((X_d, X_c), 1)
  return X, Y, mu, std

def load_test(mu, std):
  X = pd.read_csv("X_test")
  X_c = X[['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']].values
  X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss'], 1).iloc[:, :-1].values
  X_c, a, b = normalize(X_c, mu, std)
  X = np.concatenate((X_d, X_c), 1)
  return X

In [4]:
train_X, train_Y, mu, std = load_train()
print(mu)
print(std)
test_X = load_test(mu, std)

<ipython-input-3-c198a1067f96>:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss'], 1).iloc[:, :-1].values


[3.85816468e+01 1.89778367e+05 4.04374559e+01 1.07764884e+03
 8.73038297e+01]
[1.36402231e+01 1.05548357e+05 1.23472391e+01 7.38517868e+03
 4.02954031e+02]


<ipython-input-3-c198a1067f96>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_d = X.drop(['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss'], 1).iloc[:, :-1].values


In [5]:
print(test_X.shape)
print(train_X.shape)

(16281, 105)
(32561, 105)


In [6]:
#ToDo Begin
pca = PCA(n_components=16)#think what number should set
pca.fit(train_X)
train_X1 = pca.transform(train_X)


#ToDo End


In [7]:
print(train_X1.shape)

(32561, 16)


In [8]:
clf=SVC(kernel='rbf',C=1,gamma='auto')
clf.fit(train_X1, train_Y)


SVC(C=1, gamma='auto')

In [9]:
test_X1 = pca.transform(test_X)
results = clf.predict(test_X1)
with open('predict.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
    for i, x in enumerate(results):
      writer.writerow([i + 1, int(x)])

### Doing linear kernal for feature transformation output dim = 4


In [10]:
import torch
import torch.nn as nn

class FeatureTransformer(nn.Module):
    def __init__(self,output_dim, input_dim = 105 , hidden_dim = 20 ):
        super(FeatureTransformer, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [11]:
output_dim = 4
transformer = FeatureTransformer(output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001)
num_epochs = 500
tensor_X = torch.tensor(train_X).float()
pca1 = PCA(n_components=4)#think what number should set
pca1.fit(train_X)
train_X1 = pca1.transform(train_X)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = transformer(tensor_X)
    loss = criterion(output, torch.tensor(train_X1).float())  # Using MSE loss for demonstration
    loss.backward()
    optimizer.step()
    if(epoch % 100) == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")



Epoch [1/500], Loss: 1.1393
Epoch [101/500], Loss: 0.3192
Epoch [201/500], Loss: 0.0140
Epoch [301/500], Loss: 0.0028
Epoch [401/500], Loss: 0.0016


In [12]:
new_x = transformer(torch.tensor(train_X).float())
new_test = transformer(torch.tensor(test_X).float())

In [13]:
new_x = new_x.detach().numpy()
new_x.shape

(32561, 4)

In [14]:
clf_new=SVC(kernel='linear', C=1)
clf_new.fit(new_x, train_Y)

SVC(C=1, kernel='linear')

In [15]:
new_test = new_test.detach().numpy()
new_test.shape

(16281, 4)

In [16]:
results_new = clf_new.predict(new_test)
with open('predict for output = 4.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
    for i, x in enumerate(results_new):
      writer.writerow([i + 1, int(x)])

### Doing linear kernal for feature transformation output dim = 1024



In [17]:
output_dim = 1024
transformer_1024 = FeatureTransformer(output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(transformer_1024.parameters(), lr=0.001)
num_epochs = 500
np.random.seed(42)
train_x1 = np.random.rand(32561, 1024)
train_x1[:, :105] = train_X
tensor_X_1024 = torch.tensor(train_X).float()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = transformer_1024(tensor_X_1024)
    loss = criterion(output, torch.tensor(train_x1).float())  # Using MSE loss for demonstration
    loss.backward()
    optimizer.step()
    if(epoch % 100) == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/500], Loss: 0.3332
Epoch [101/500], Loss: 0.0875
Epoch [201/500], Loss: 0.0837
Epoch [301/500], Loss: 0.0825
Epoch [401/500], Loss: 0.0815


In [18]:
new_x = transformer_1024(torch.tensor(train_X).float())
new_test = transformer_1024(torch.tensor(test_X).float())
new_x = new_x.detach().numpy()
new_test = new_test.detach().numpy()
new_x.shape, new_test.shape

((32561, 1024), (16281, 1024))

In [ ]:
clf_new_1024=SVC(kernel='linear', C=1)
clf_new_1024.fit(new_x, train_Y)

In [ ]:
results_new_1024 = clf_new_1024.predict(new_test)
with open('predict for output = 1024.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id','label'])
    for i, x in enumerate(results_new_1024):
      writer.writerow([i + 1, int(x)])